In [19]:
import pandas as pd
import plotly.express as px
import plotly

In [6]:
admissions = pd.read_csv('data-ingest/data/admissions_hb_long_unfiltered.csv', parse_dates = [0])
admissions.head()

,date,health_board,admission_type,count
0,2020-04-01,Betsi Cadwaladr University Health Board,C19,28
1,2020-04-01,Betsi Cadwaladr University Health Board,NonC19,98
2,2020-04-01,Betsi Cadwaladr University Health Board,All,126
3,2020-04-01,Hywel Dda University Health Board,C19,16
4,2020-04-01,Hywel Dda University Health Board,NonC19,63


In [7]:
admissions.health_board.unique()

array(['Betsi Cadwaladr University Health Board',
       'Hywel Dda University Health Board',
       'Abertawe Bro Morgannwg University Health Board',
       'Cardiff and Vale University Health Board',
       'Cwm Taf University Health Board',
       'Aneurin Bevan University Health Board',
       'Velindre University NHS Trust', 'Wales',
       'Powys Teaching Health Board'], dtype=object)

In [9]:
admissions_covid = admissions[admissions['admission_type'] == 'C19'].drop(columns = ['admission_type'])
admissions_covid.tail()

,date,health_board,count
6021,2020-11-12,Cwm Taf University Health Board,15
6024,2020-11-12,Aneurin Bevan University Health Board,20
6027,2020-11-12,Powys Teaching Health Board,0
6030,2020-11-12,Velindre University NHS Trust,0
6033,2020-11-12,Wales,100


In [12]:
lockdowns = pd.read_csv('data-ingest/data/lockdown_status_hb_date.csv', parse_dates = [0])
lockdowns['date'] = pd.to_datetime(lockdowns['date'], format = '%d/%m/%Y')
lockdowns.tail()

,date,health_board,lockdown_status
1998,2020-07-11,Wales,No status
1999,2020-08-11,Wales,No status
2000,2020-09-11,Wales,No status
2001,2020-10-11,Wales,No status
2002,2020-11-11,Wales,No status


In [13]:
admss_lockdowns = admissions_covid.merge(lockdowns, how = 'inner', on = ['date', 'health_board'])
admss_lockdowns.head()

,date,health_board,count,lockdown_status
0,2020-04-04,Betsi Cadwaladr University Health Board,30,Full national lockdown
1,2020-04-04,Hywel Dda University Health Board,7,Full national lockdown
2,2020-04-04,Abertawe Bro Morgannwg University Health Board,24,Full national lockdown
3,2020-04-04,Cardiff and Vale University Health Board,30,Full national lockdown
4,2020-04-04,Cwm Taf University Health Board,30,Full national lockdown


In [21]:
all_wales = admss_lockdowns[admss_lockdowns['health_board']=='Wales']

fig = px.line(all_wales, x = 'date', y = 'count')
plotly.offline.plot(fig)

'temp-plot.html'

In [24]:
# find the first day of local lockdowns for all healthboards.

def lockdown_countdown(df: pd.DataFrame, hb: str) -> pd.DataFrame:
    temp_df = df[df['health_board'] == hb].sort_values('date')
    first_day = temp_df[temp_df['lockdown_status'] != 'No local lockdowns'].iloc[0,0]
    temp_df = temp_df[temp_df['date']>= first_day - pd.Timedelta('28 days')]
    temp_df = temp_df[temp_df['lockdown_status'] != 'Firebreak lockdown'] 
    temp_df['daycount_from_local_lockdown'] = list(range(-28, len(temp_df)-28))
    return temp_df

countdown_df = pd.concat([lockdown_countdown(admss_lockdowns, hb) for hb in admss_lockdowns.health_board.unique() if hb not in ['Powys Teaching Health Board', 'Velindre University NHS Trust', 'Wales']])
countdown_df.tail()

,date,health_board,count,lockdown_status,daycount_from_local_lockdown
1681,2020-11-07,Aneurin Bevan University Health Board,50,No local lockdowns,147
1690,2020-11-08,Aneurin Bevan University Health Board,20,No local lockdowns,148
1699,2020-11-09,Aneurin Bevan University Health Board,9,Local lockdowns,149
1708,2020-11-10,Aneurin Bevan University Health Board,23,Local lockdowns,150
1717,2020-11-11,Aneurin Bevan University Health Board,29,Post-firebreak,151


In [27]:
#all_wales = beds_lockdowns[beds_lockdowns['health_board']=='Wales']

fig = px.line(countdown_df[countdown_df['daycount_from_local_lockdown'] < 56], 
              x = 'daycount_from_local_lockdown', y = 'count', color = 'health_board')
plotly.offline.plot(fig)

'temp-plot.html'

In [30]:
hbs_filterable = countdown_df[countdown_df['daycount_from_local_lockdown'] < 56].drop(columns = ['date', 'lockdown_status'])
hbs_filterable = hbs_filterable[['daycount_from_local_lockdown', 'health_board', 'count']]
hbs_filterable.head()

,daycount_from_local_lockdown,health_board,count
0,-28,Betsi Cadwaladr University Health Board,30
8,-27,Betsi Cadwaladr University Health Board,23
16,-26,Betsi Cadwaladr University Health Board,19
24,-25,Betsi Cadwaladr University Health Board,28
32,-24,Betsi Cadwaladr University Health Board,32


In [35]:
hbs_filterable['ravg_admissions'] = hbs_filterable.groupby('health_board')['count'].transform(lambda x: x.rolling(7, 1).mean())
hbs_filterable.head()

,daycount_from_local_lockdown,health_board,count,ravg_admissions
0,-28,Betsi Cadwaladr University Health Board,30,30.0
8,-27,Betsi Cadwaladr University Health Board,23,26.5
16,-26,Betsi Cadwaladr University Health Board,19,24.0
24,-25,Betsi Cadwaladr University Health Board,28,25.0
32,-24,Betsi Cadwaladr University Health Board,32,26.4


In [36]:
hbs_columns = hbs_filterable.pivot(index = 'daycount_from_local_lockdown', columns = 'health_board', values = 'ravg_admissions') 
hbs_columns.head()

health_board,Abertawe Bro Morgannwg University Health Board,Aneurin Bevan University Health Board,Betsi Cadwaladr University Health Board,Cardiff and Vale University Health Board,Cwm Taf University Health Board,Hywel Dda University Health Board
daycount_from_local_lockdown,,,,,,
-28,24.000000,13.000000,30.0,30.0,30.000000,7.00
-27,19.500000,21.500000,26.5,23.0,32.500000,11.00
-26,18.666667,23.333333,24.0,20.0,31.333333,13.00
-25,20.750000,33.250000,25.0,20.5,33.750000,13.25
-24,23.800000,31.400000,26.4,21.6,32.000000,13.60


In [39]:
import plotly.graph_objects as go
fig = go.Figure()

for column in hbs_columns.columns.to_list():
    fig.add_trace(
        go.Scatter(
            x = hbs_columns.index,
            y = hbs_columns[column],
            name = column
        )
    )
    
main_title = 'Rolling average (over one week) of daily hospital admissions<br>Health board:  '
fig.update_layout(
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=list(
            [dict(label = 'All',
                  method = 'update',
                  args = [{'visible': [True, True, True, True, True, True]},
                          {'title': main_title + 'All',
                           'showlegend':True}]),
             dict(label = 'Abertawe Bro Morgannwg',
                  method = 'update',
                  args = [{'visible': [True, False, False, False, False, False]}, # the index of True aligns with the indices of plot traces
                          {'title': main_title + 'Abertawe Bro Morgannwg',
                           'showlegend':True}]),
             dict(label = 'Aneurin Bevan',
                  method = 'update',
                  args = [{'visible': [False, True, False, False, False, False]},
                          {'title': main_title + 'Aneurin Bevan',
                           'showlegend':True}]),
             dict(label = 'Betsi Cadwaladr',
                  method = 'update',
                  args = [{'visible': [False, False, True, False, False, False]},
                          {'title': main_title + 'Betsi Cadwaladr',
                           'showlegend':True}]),
             dict(label = 'Cardiff and Vale',
                  method = 'update',
                  args = [{'visible': [False, False, False, True, False, False]},
                          {'title': main_title + 'Cardiff and Vale',
                           'showlegend':True}]),
             dict(label = 'Cwm Taf',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, True, False]},
                          {'title': main_title + 'Cwm Taf',
                           'showlegend':True}]),
             dict(label = 'Hywel Dda',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, False, True]},
                          {'title': main_title + 'Hywel Dda',
                           'showlegend':True}]),
            ])
        )
    ])

fig.write_html('visualisations/lockdown_countdown_admissions.html')
fig.show()